In [35]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
from pathlib import Path

# === SESSION SETUP ===
# Set project + data directories (defined by user)
BASE_PROJECT_DIR = Path("/content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project")  # Output location
RAW_DIR = Path("/content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/SIMPLEthelia_project/data_raw_simplethelia")              # Input location (experiments)

# === SEGMENTATION PARAMETERS ===
Z_AXIS = 0
CHANNEL_AXIS = 1
BATCH_SIZE = 32
DO_3D = False
STITCH_THRESHOLD = 0.5




In [37]:
# Install latest Cellpose from GitHub (includes SAM support)
!pip install git+https://github.com/mouseland/cellpose.git

  Cloning https://github.com/mouseland/cellpose.git to /tmp/pip-req-build-72uq8i_a
  Running command git clone --filter=blob:none --quiet https://github.com/mouseland/cellpose.git /tmp/pip-req-build-72uq8i_a
  Resolved https://github.com/mouseland/cellpose.git to commit df6b944673c3ab6d46c2cb780bcde040970dbebc
  Preparing metadata (setup.py) ... done


In [42]:
# from pathlib import Path

# # Adjust this to your actual full path to lib/
# lib_path = Path("/content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/notebooks/lib")

# print(" lib_path exists:", lib_path.exists())
# print(" Contents of lib/:")
# for f in lib_path.glob("*"):
#     print("  ", f.name)


✅ lib_path exists: True
📁 Contents of lib/:
  └── segmentation.py
  └── __pycache__


In [43]:
# import sys
# import importlib

# # Add the lib directory to Python's path
# lib_path = "/content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/notebooks/lib"
# if lib_path not in sys.path:
#     sys.path.insert(0, lib_path)

# # Import segmentation.py
# import segmentation
# importlib.reload(segmentation)  # Make sure it’s reloaded freshly

# # Optional: list all available functions
# print(" Functions available in segmentation.py:")
# for f in dir(segmentation):
#     if not f.startswith("_"):
#         print("  ", f)


📦 Functions available in segmentation.py:
  └── Path
  └── get_pickle_groups_by_experiment
  └── get_tiff_groups_by_experiment
  └── imread
  └── imwrite
  └── np
  └── pickle
  └── print_experiment_summary
  └── run_segmentation_pipeline
  └── save_segmentation_pickle
  └── save_segmentation_tiff
  └── segment_and_save
  └── segment_single_tiff


In [44]:
# # Replace with your actual raw data path
# RAW_DIR = Path("/content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/SIMPLEthelia_project/data_raw_simplethelia")

# # Run the summary
# segmentation.print_experiment_summary(RAW_DIR)



🔍 Scanning raw data directory: /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/SIMPLEthelia_project/data_raw_simplethelia
🧪 Found 6 TIFF(s) in ITB1cell3:
   └── ITB1cell3.oif - Series 1-1-1.tif
   └── ITB1cell3.oif - Series 1-1-5.tif
   └── ITB1cell3.oif - Series 1-1-2.tif
   └── ITB1cell3.oif - Series 1-1-6.tif
   └── ITB1cell3.oif - Series 1-1-4.tif
   └── ITB1cell3.oif - Series 1-1-3.tif
🔎 Found 1 experiment folder(s) to process.
  1. ITB1cell3: 6 TIFF(s)
    📐 Shape: (60, 452, 450)


In [45]:
# === SESSION SETUP IMPORTS AND FUNCTIONS ===
# Part 1: Define directories, discover files, and run segmentation

import sys
import numpy as np
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import trange
import matplotlib.pyplot as plt
from tifffile import imread
from tifffile import imwrite  #  use imwrite, not imsave
import pickle
from datetime import datetime
import importlib

# === SESSION SETUP: Add lib/ to path and import segmentation.py ===
LIB_DIR = BASE_PROJECT_DIR / "notebooks" / "lib"
if str(LIB_DIR) not in sys.path:
    sys.path.insert(0, str(LIB_DIR))

import segmentation
importlib.reload(segmentation)  #  Fresh reload if module already loaded

#  List available functions
print(" Functions available in segmentation.py:")
for f in dir(segmentation):
    if not f.startswith("_"):
        print("  ", f)

# === FURTHER SESSION SETUP ===

OUTPUTS_DIR = BASE_PROJECT_DIR / "outputs"
OUTPUTS_DIR.mkdir(exist_ok=True)

if not RAW_DIR.exists():
    raise FileNotFoundError(f" Raw input folder not found: {RAW_DIR}")

print(f" Project outputs will be saved to: {OUTPUTS_DIR}")
print(f" Looking for raw experiment folders in: {RAW_DIR}")

io.logger_setup()  # Enables progress printing

# Check if GPU is available
if not core.use_gpu():
    raise ImportError(" No GPU access. Change your Colab runtime to GPU.")

# Load Cellpose model (SAM backend)
model = models.CellposeModel(gpu=True)

📦 Functions available in segmentation.py:
  └── Path
  └── get_pickle_groups_by_experiment
  └── get_tiff_groups_by_experiment
  └── imread
  └── imwrite
  └── np
  └── pickle
  └── print_experiment_summary
  └── run_segmentation_pipeline
  └── save_segmentation_pickle
  └── save_segmentation_tiff
  └── segment_and_save
  └── segment_single_tiff
📁 Project outputs will be saved to: /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs
🔍 Looking for raw experiment folders in: /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/SIMPLEthelia_project/data_raw_simplethelia
2025-07-24 19:11:06,093 [INFO] WRITING LOG OUTPUT TO /root/.cellpose/run.log
2025-07-24 19:11:06,094 [INFO] 
cellpose version: 	4.0.7.dev7+gdf6b944 
platform:       	linux 
python version: 	3.11.13 
torch version:  	2.6.0+cu124
2025-07-24 19:11:06,096 [INFO] ** TORCH CUDA version installed and working. **
2025-07-24 19:11:06,097 [INFO] ** TORCH CUDA version installed and workin

In [46]:
# === RUN DISCOVERY SUMMARY FUNCTION ===
print_experiment_summary(RAW_DIR)



🔍 Scanning raw data directory: /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/SIMPLEthelia_project/data_raw_simplethelia
🧪 Found 6 TIFF(s) in ITB1cell3:
   └── ITB1cell3.oif - Series 1-1-1.tif
   └── ITB1cell3.oif - Series 1-1-5.tif
   └── ITB1cell3.oif - Series 1-1-2.tif
   └── ITB1cell3.oif - Series 1-1-6.tif
   └── ITB1cell3.oif - Series 1-1-4.tif
   └── ITB1cell3.oif - Series 1-1-3.tif
🔎 Found 1 experiment folder(s) to process.
  1. ITB1cell3: 6 TIFF(s)
    📐 Shape: (60, 452, 450)


In [48]:
# === RUN PIPELINE ===
run_segmentation_pipeline(
    RAW_DIR,
    OUTPUTS_DIR,
    model,
    z_axis=Z_AXIS,
    channel_axis=CHANNEL_AXIS,
    batch_size=BATCH_SIZE,
    do_3D=DO_3D,
    stitch_threshold=STITCH_THRESHOLD
)



🔍 Scanning raw data directory: /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/SIMPLEthelia_project/data_raw_simplethelia
🧪 Found 6 TIFF(s) in ITB1cell3:
   └── ITB1cell3.oif - Series 1-1-1.tif
   └── ITB1cell3.oif - Series 1-1-5.tif
   └── ITB1cell3.oif - Series 1-1-2.tif
   └── ITB1cell3.oif - Series 1-1-6.tif
   └── ITB1cell3.oif - Series 1-1-4.tif
   └── ITB1cell3.oif - Series 1-1-3.tif

🚀 Starting segmentation for experiment: ITB1cell3 (6 TIFFs)
📊 Will now segment 6 TIFF file(s)...
📂 Segmenting: ITB1cell3.oif - Series 1-1-1.tif
2025-07-24 19:11:44,392 [INFO] 100%|##########| 20/20 [00:08<00:00,  2.40it/s]
2025-07-24 19:11:44,393 [INFO] network run in 8.32s
2025-07-24 19:11:44,622 [INFO] 0%|          | 0/60 [00:00<?, ?it/s]
2025-07-24 19:11:44,623 [INFO] No cell pixels found.
2025-07-24 19:11:49,055 [INFO] No cell pixels found.
2025-07-24 19:11:49,133 [INFO] No cell pixels found.
2025-07-24 19:11:49,136 [INFO] No cell pixels found.
2025-07-24 19:11:49,138 [INFO] No c

100%|██████████| 59/59 [00:00<00:00, 202.55it/s]


2025-07-24 19:11:50,557 [INFO] masks created in 5.94s
✅ Saved: segmented_ITB1cell3.oif - Series 1-1-1.pkl to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_pickles
✅ Saved TIFF: segmented_ITB1cell3.oif - Series 1-1-1.tif to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_tiffs
📂 Segmenting: ITB1cell3.oif - Series 1-1-2.tif
2025-07-24 19:11:59,556 [INFO] 100%|##########| 20/20 [00:07<00:00,  2.59it/s]
2025-07-24 19:11:59,556 [INFO] network run in 7.74s
2025-07-24 19:11:59,752 [INFO] 0%|          | 0/60 [00:00<?, ?it/s]
2025-07-24 19:11:59,753 [INFO] No cell pixels found.
2025-07-24 19:11:59,759 [INFO] No cell pixels found.
2025-07-24 19:12:04,103 [INFO] No cell pixels found.
2025-07-24 19:12:04,104 [INFO] No cell pixels found.
2025-07-24 19:12:04,106 [INFO] No cell pixels found.
2025-07-24 19:12:04,108 [INFO] No cell pixels found.
2025-07-2

100%|██████████| 59/59 [00:00<00:00, 204.70it/s]


2025-07-24 19:12:05,027 [INFO] masks created in 5.27s
✅ Saved: segmented_ITB1cell3.oif - Series 1-1-2.pkl to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_pickles
✅ Saved TIFF: segmented_ITB1cell3.oif - Series 1-1-2.tif to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_tiffs
📂 Segmenting: ITB1cell3.oif - Series 1-1-3.tif
2025-07-24 19:12:14,004 [INFO] 100%|##########| 20/20 [00:07<00:00,  2.59it/s]
2025-07-24 19:12:14,005 [INFO] network run in 7.74s
2025-07-24 19:12:14,203 [INFO] 0%|          | 0/60 [00:00<?, ?it/s]
2025-07-24 19:12:14,204 [INFO] No cell pixels found.
2025-07-24 19:12:14,210 [INFO] No cell pixels found.
2025-07-24 19:12:18,708 [INFO] No cell pixels found.
2025-07-24 19:12:18,710 [INFO] No cell pixels found.
2025-07-24 19:12:18,712 [INFO] No cell pixels found.
2025-07-24 19:12:18,714 [INFO] No cell pixels found.
2025-07-2

...

2025-07-24 19:12:34,171 [INFO] masks created in 5.36s
✅ Saved: segmented_ITB1cell3.oif - Series 1-1-4.pkl to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_pickles
✅ Saved TIFF: segmented_ITB1cell3.oif - Series 1-1-4.tif to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_tiffs
📂 Segmenting: ITB1cell3.oif - Series 1-1-5.tif
2025-07-24 19:12:43,166 [INFO] 100%|##########| 20/20 [00:07<00:00,  2.57it/s]
2025-07-24 19:12:43,167 [INFO] network run in 7.79s
2025-07-24 19:12:43,365 [INFO] 0%|          | 0/60 [00:00<?, ?it/s]
2025-07-24 19:12:43,366 [INFO] No cell pixels found.
2025-07-24 19:12:43,372 [INFO] No cell pixels found.
2025-07-24 19:12:43,374 [INFO] No cell pixels found.
2025-07-24 19:12:43,375 [INFO] No cell pixels found.
2025-07-24 19:12:43,377 [INFO] No cell pixels found.
2025-07-24 19:12:43,379 [INFO] No cell pixels found.
2025-07-2

100%|██████████| 59/59 [00:00<00:00, 195.07it/s]


2025-07-24 19:12:48,715 [INFO] masks created in 5.35s
✅ Saved: segmented_ITB1cell3.oif - Series 1-1-5.pkl to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_pickles
✅ Saved TIFF: segmented_ITB1cell3.oif - Series 1-1-5.tif to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_tiffs
📂 Segmenting: ITB1cell3.oif - Series 1-1-6.tif
2025-07-24 19:12:57,647 [INFO] 100%|##########| 20/20 [00:07<00:00,  2.59it/s]
2025-07-24 19:12:57,648 [INFO] network run in 7.71s
2025-07-24 19:12:57,842 [INFO] 0%|          | 0/60 [00:00<?, ?it/s]
2025-07-24 19:12:57,844 [INFO] No cell pixels found.
2025-07-24 19:12:57,849 [INFO] No cell pixels found.
2025-07-24 19:12:57,851 [INFO] No cell pixels found.
2025-07-24 19:12:57,853 [INFO] No cell pixels found.
2025-07-24 19:12:57,854 [INFO] No cell pixels found.
2025-07-24 19:12:57,856 [INFO] No cell pixels found.
2025-07-2

100%|██████████| 59/59 [00:00<00:00, 212.55it/s]


2025-07-24 19:13:03,328 [INFO] masks created in 5.49s
✅ Saved: segmented_ITB1cell3.oif - Series 1-1-6.pkl to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_pickles
✅ Saved TIFF: segmented_ITB1cell3.oif - Series 1-1-6.tif to /content/drive/MyDrive/MP_Epithelia_2025/cellposeSAM_mount_LD/Hyperthelia_project/outputs/outputs_ITB1cell3/raw_segmented_tiffs
